# Connecting Kamodo to PYGEODYN

This notebook details the processes for connecting `Kamodo` to `PYGEODYN`.

## Overview of Steps
1. Download newest version of Kamodo (https://github.com/rebeccaringuette/Kamodo)
    - ``git clone https://github.com/rebeccaringuette/Kamodo``
2. Access to Kamodo thru Pygeodyn will occur in the `DRAG.f90` subroutine.  We will additionally construct a KamodoModel.f90 subroutine to actually call the Kamodo models.
    - Kamodo models will be called through the command line within the fortran executable
    - The command line call will write the data to a text file.
    - The `KamodoModel.f90` subroutine reads the text file back into the GEODYN code to get passed back to Drag.f90.
    
    ```
    ├──── DRAG.f90 
    |    └──────> KamodoModel.f90 
    |           └──────> cmdline 
    |                  └──────> Kamodo/SingleSatelliteFlythrough.py
    |                         └──────> text file
    |
    ├───(read textfile)─── KamodoModel.f90  
    ├──────(return)────── DRAG.f90 
    ```
    
3. 

### Tracking changes to Kamodo:

- made a separate directory containing the relevant files and my changes to them
 - add the path of kamodo codes to `SingleSatelliteFlythrough.py`
     ```python
     import sys
     sys.path.insert(0,'/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/')
     ```
     
 - remove importing of any plots GEODYN case. (in `SatelliteFlythrough.py`)  
 
    -`SatelliteFlythrough.py` 
     ``` 
        import os
        import numpy as np
        from kamodo.readers.hapi import HAPI
        from spacepy import coordinates as coord
        from spacepy.time import Ticktock
        from astropy.constants import R_earth
        # from kamodo.satelliteflythrough import FlythroughPlots as FPlot
        from kamodo.satelliteflythrough.wrapper_output import SFdata_tocsv
        import kamodo.satelliteflythrough.wrapper_utilities as U
     ```
    
    -`wrapper_utilities.py` 
     ``` 
      import glob, os
      import numpy as np
      import time as ti
      from datetime import datetime, timedelta, timezone
      import kamodo.satelliteflythrough.model_wrapper as MW
      # import kamodo.satelliteflythrough.FlythroughPlots as FPlot
      from pprint import pprint     
      ```

- in ``model_wrapper.py``, remove `Data/` from path of the `FileSearch()` function.
- add an empty ``init__.py to the pygeodyn_connection directory

### Tracking changes/considerations in pygeodyn code:

1. Add new models to be accepted as inputs to pygeodyn.
 - in `/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/util_classtools.py`, add the additional models to the `geodyn_modify_inputs()` func.  This writes the file that is read into geodyn at each runtime with the model options.
    ```python
       def geodyn_modify_inputs(self, options_in, density_model):
           if options_in['DRHODZ_update']== True:
                drhodz_val = '1'
           elif options_in['DRHODZ_update']== False:
                drhodz_val = '0'
           else:
                sys.exit("DRHODZ option is in incorrect format")
           if density_model== 'msis86':
                model_val = '0'
           elif density_model== 'msis00':
                model_val = '1'
           elif density_model== 'msis2':
                model_val = '2'
           elif density_model== 'ctipe':
                model_val = '3'
           elif density_model== 'tiegcm':
                model_val = '4'
           elif density_model== 'gitm':
                model_val = '5'
           elif density_model== 'jaachia71':
                model_val = '0'        
           elif density_model== 'dtm87':
                model_val = '0'
           else:
                sys.exit("Density Model Option (DEN_DIR) is in incorrect format")

           file1 = open("/data/geodyn_proj/pygeodyn/pygeodyn_develop/geodyn_options.txt","w+")
           file1.writelines(drhodz_val+'\n') # first value is for DrhoDz
           file1.writelines(model_val +'\n') # 2nd values is for model switching
           file1.writelines('0'+'\n')
           file1.writelines('0'+'\n')
           file1.writelines('0'+'\n')
           file1.writelines('0'+'\n')
           file1.close()
        ```
        
  - add model cases to ``set_density_model_setup_params()``
2. Add new models to `DRAG.f90`
  - ``/data/geodyn_proj/geodyn_code/IIE/Kamodo_pygeodyn_MODS/DRAG.f90``
  - add a case such that values greater than 3 call KamodoModels.f90.  add conditionals that identify the model.
      ```fortran
              case(3:)   ! Use Kamodo if case is greaterthan or equal to one
                 
                 if(choose_model.eq.3)then
                     kamodo_model='CTIPE'
                 if(choose_model.eq.4)then
                     kamodo_model='TIEGCM'
                 if(choose_model.eq.5)then
                     kamodo_model='GITM'
                 endif
                 
                 !!!!! Inputs for the Kamodo Reader
                 !          File_dir is the directory where the data is located
                 !          rho is the variable name
                 !          ilev means the variable depends on the pressure level for the CTIPe and TIEGCM model.
                 !          sat_time is the satellite timestamp in UTC since Jan 1 1970
                 !          sat_height: is satellite altitude above the ground (in km)
                 !          sat_lat satellite latitude
                 !          sat_lon satellite longitude               
                
                 !   sat_time   -- !   MJDSEC   I    S    TIME IN INTEGRAL SECONDS FROM GEODYN REF. TIME (
                                   ! will need to correct difference in ref times
                 !   sat_height -- ALTI
                 !   sat_lat    -- XLATD
                 !   sat_lon    -- XLOND
!                                                                       &
                if(kentry.eq.1)then
                  WRITE(6,*) 'CHECK-- DRAG.f90: Running w/ KamodoModels'
                endif
!                                                                       &
                 IJDSEC=MJDSEC+FSEC
                 CALL MJDYMD(IJDSEC,IYMD,IHMS,4)
                 IYR=(IYMD/10000.D0)+0.5D0
                CALL KamodoModels(MJDSEC,FSEC,ALTI,XLATD,XLOND,         &
             &         PHI,XLAMB,RHO,DRHODZ, DAY,IYR)
                C(1)=DRHODZ          
                if(kentry.eq.1)then
                  WRITE(6,*) 'CHECK-- DRAG.f90: Running w/ KamodoModels'
                endif
          end select
      ```
      
      
2. Made many changes to ``KamodoModels.f90``. It is commented, use it as a template.

3. Needed to split up the sat_time input in ``Kamodo/kamodo/satelliteflythrough/SingleSatelliteFlythrough.py``:
    ```python
        YYMMDD = str(sys.argv[6])
        HHMMSS = str(sys.argv[7])
        sat_time_dt = pd.to_datetime(YYMMDD+HHMMSS , format='%y%m%d%H%M%S')
        sat_time = time.mktime(sat_time_dt.timetuple())  
        # sat_time   = float(sys.argv[6]) #1426637500.0
        sat_height = float(sys.argv[8]) #400. (in km)
    ```
    
    
4. Changed verbose to be false in wrapper_utilities.py (Calcilev)


### Some tests:

##### DZ numerical computation
- DZ on (1)
   - 3.38 s ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
- DZ off (0)
   - 3.38 s ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
##### This is what made it faster: in `kamodo/pygeodyn_readers/ctipe_4D.py`
  - remove all references to the three other CTIPE files 
  - Turn off the `gridded_int` option when calling the interpolators (cuts time by like 75%)

## Test Kamodo calls outside of GEODYN

We are calling Kamodo thru the command line using RR's `SingleSatelliteFlythrough.py` program.
  - program makes call for a single trajectory point from the command line
 


#### Model pre-run


In [1]:
# %load_ext autoreload
# %autoreload 2
# import os

# kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/satelliteflythrough/SingleSatelliteFlythrough.py'
# model= 'CTIPe'
# file_dir     = '/data/data_geodyn/atmos_models_data/ctipe/2015_03_18/'
# # arg 3
# variable_list = "['rho']"
# z_dependence  = "['ilev']"
# dz            = "['1']"            # 1 request partial derivative of variable 0 is no calculation
# # arg 6
# sat_time      = '1426637500.0' # unix time
# sat_height    = '400.0'        # altitude in km
# sat_lat       = '-25.'         # range of -90 to +90 degrees
# sat_lon       = '90.'          # range of 0 to 360 degrees
# model_dt      = '450.'         # model_dt: half of the time resolution of the model data, given by the output 
#                                #           of the third syntax choice. This value is used as a maximum allowed time 
#                                #           shift for times requested that are between the model data files.
# high_res      = '0.02'         # precision of plevel interp in meters

# command = ('python'             +' '+
#             kamodo_program_path +' '+
#             model               +' '+          
#             file_dir            +' '+
#             variable_list      
# #             z_dependence        +' '+
# #             dz                  +' '+
# #             sat_time            +' '+
# #             sat_height          +' '+
# #             sat_lat             +' '+
# #             sat_lon             +' '+
# #             model_dt            +' '+
# #             high_res 
#           )
# print(command)
# os.system(command)


#### Sample satellite density

In [2]:
# import os

# # command = 'python /data/geodyn_proj/interface_kamodo_geodyn/Kamodo-master/kamodo/readers/CTIPe_wrapper_fortrancmd.py /data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/ rho ilev 181201 210519 485.4546760656302808    40.18117715905883  -117.07021273480323'
# #
# ### Prepare the command line entry:
# ####        >>> python 
# ####            program_path/program_name.py 
# ####            model data path 
# ####            file_dir 
# ####            variable_list  
# ####            z_dependence 
# ####            dz 
# ####            sat_time 
# ####            sat_height 
# ####            sat_lat 
# ####            sat_lon 
# ####            model_dt 
# ####            high_res

# kamodo_program_path = '/data/geodyn_proj/interface_kamodo_geodyn/Kamodo/kamodo/satelliteflythrough/SingleSatelliteFlythrough.py'
# model= 'CTIPe'
# file_dir     = '/data/data_geodyn/atmos_models_data/ctipe/2018_Dec_1_15/'
# # arg 3
# variable_list = "['rho']"
# z_dependence  = "['ilev']"
# dz            = "['1']"            # 1 request partial derivative of variable 0 is no calculation
# # arg 6
# sat_time1      = '181202'#YYMMDD   #'1426637500.0' # unix time
# sat_time2      = '121014'#HHMMSS
# sat_height    = '300.0'        # altitude in km
# sat_lat       = '-25.'         # range of -90 to +90 degrees
# sat_lon       = '90.'          # range of 0 to 360 degrees
# model_dt      = '450.'         # model_dt: half of the time resolution of the model data, given by the output 
#                                #           of the third syntax choice. This value is used as a maximum allowed time 
#                                #           shift for times requested that are between the model data files.
# high_res      = '10.0'         # precision of plevel interp in meters

# command = ('python'             +' '+
#             kamodo_program_path +' '+
#             model               +' '+
#             file_dir            +' '+
#             variable_list       +' '+
#             z_dependence        +' '+
#             dz                  +' '+
#             sat_time1            +' '+
#             sat_time2            +' '+
#             sat_height          +' '+
#             sat_lat             +' '+
#             sat_lon             +' '+
#             model_dt            +' '+
#             high_res 
#            )

# %timeit os.system(command)

# # with open(file_dir+model+'_results.txt', 'r') as f:
# #     for line_no, line in enumerate(f):
# #         print(line)


## Adding TIEGCM to pygeodyn-Kamodo

1. Starting at the `Pygeodyn` call
 - Calling pygeodyn with the TIEGCM model tells Pygeodyn to write the `geodyn_options.txt` with a `4` in the model line (line 2).  

2. `DRAG.f90`
 - This text file is then read in by `DRAG.f90` at the time of execution as the variable `optionsin(2)` before being saved as `choose_model`. 
 - The setup file will be written with card ATMDEN=86 (This is MSISe86 but I'm using its settings as the default to call Kamodo)
 - The select case conditional will be activated for the the case where `choose_model>3`.  This is the Kamodo Case.  We then identify which model it is based on its exact value (`tiegcm=4`).  Kamodo_model stores the string `TIEGCM` to be used in `KamodoModels.f90` and the Kamodo Python code.  

3. `KamodoModels.f90`
 - From `DRAG.f90`, call `KamodoModels.f90` with the MJDSEC, FSEC, ALTM, XLATD, XLOND, PHI, XLAMB, RHO, DRHODZ, DAY, IYR, kamodo_model
 - 

## Test with Pygeodyn
Test with the icesat2 configuration of pygeodyn

```
    Date       DOY      Epoch in iisset
    -------------------------------------
    Nov 29     333A     not used
    Nov 30     334    
    Dec 1      335        
    Dec 2      336      2018/12/01 21:00   -- 2018/12/04/ 03:00
    Dec 3      337   
    Dec 4      338    
    Dec 5      339A     not used
```

In [3]:
%load_ext autoreload
%autoreload 2

# import copy
import sys
import os.path
import numpy as np

#------ A dictionary containing the run parameters ------  
run_params = {} 
run_params['arc']              =   ['2018.336']
run_params['satellite']        =  'icesat2'  
run_params['den_model']        =  'ctipe'  
run_params['SpecialRun_name']  =  '_testkamodo_w_'+run_params['den_model']  
run_params['verbose']          =  False
run_params['action']           =  'run'



sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

sys.path.insert(0, '/data/geodyn_proj/interface_kamodo_geodyn/modify_icesat2_setup')
from modify_icesat2_setup import edit_ICESat2_setup


### Run pyeodyn for the arcs in the above set.
# Obj_Geodyn = Pygeodyn(run_params)
# Obj_Geodyn.RUN_GEODYN()

# Construct object with run params
Obj_Geodyn = edit_ICESat2_setup(run_params)
# Run pyeodyn for the arcs in the above set.
Obj_Geodyn.RUN_GEODYN()


                      ......... RUNNING GEODYN
Run # 1     Current Time =      20:06:35
Run # 1
Run # 1     Cleaning iisset... :    /data/data_geodyn/inputs/icesat2/setups/iisset.2018.336.bz2
Epoch Start:  2018-12-01 21:00:00
epoch end   : 1812040300 0
epoch start : 181201210000.00
181202060000
181202150000
181203000000
181203090000
181203180000
181204030000
+ ——————————————————————————————————————————————————————————————————————————————————————————————————————————————
|
| ---------------------- Some run information ----------------------
|
|  Run # 1     IISSET Cleaned      tmp/.../cleaned_setup_2018336
|  Run # 1     Density Model:      ctipe
|  Run # 1     GEODYN Version:     Kamodo_pygeodyn_MODS
|  Run # 1     ARC run:            icesat2_2018336_54hr.ctipe
|  Run # 1     Output directory:   /data/data_geodyn/results/icesat2/ctipe/ctipe_acceloff_testkamodo_w_ctipe
|  Run # 1     EXAT File:          /data/data_geodyn/inputs/icesat2/external_attitude/EXAT01.2018.336.gz
|
+ ———————————

KeyboardInterrupt: 